In [1]:
!pip install numpy
!pip install tensorflow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import Recall, Accuracy, Precision
from tensorflow.keras import backend as K

# Definir una función personalizada para calcular F1-Score
def f1_score(y_true, y_pred):
    precision = Precision()(y_true, y_pred)
    recall = Recall()(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Ruta a los datos de entrenamiento y validación
train_data_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/platos_de_comida/train'
validation_data_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/platos_de_comida/validation'
model_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/Modelo/food_classification_model_5.h5'

# Preprocesamiento de datos y aumentación de imágenes
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.1
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Cargar el modelo existente o crear uno nuevo si no existe
if os.path.exists(model_path):
    print("Cargando modelo existente...")
    model = load_model(model_path, custom_objects={'f1_score': f1_score})
    # Si se agregan nuevas clases, ajustar las últimas capas
    if model.layers[-1].units != train_generator.num_classes:
        x = model.layers[-4].output  # obtener la salida de la capa antes de Dense
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        output = Dense(train_generator.num_classes, activation='softmax')(x)
        model = Model(inputs=model.input, outputs=output)
        # Compilar el modelo con las nuevas capas
        model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', Recall(), Precision(), f1_score])
else:
    print("Creando un modelo nuevo...")
    # Cargar el modelo VGG16 preentrenado sin las capas superiores (fully connected layers)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Congelar las capas del modelo base
    for layer in base_model.layers:
        layer.trainable = False

    # Añadir nuevas capas superiores para nuestro problema específico
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(train_generator.num_classes, activation='softmax')(x)

    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=output)

    # Compilar el modelo
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', Recall(), Precision(), f1_score])

# Configurar el callback para guardar el mejor modelo basado en la métrica de validación de precisión (val_accuracy)
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

# Cargar el mejor modelo guardado
model = load_model(model_path, custom_objects={'f1_score': f1_score})

# Evaluar el modelo final en los datos de validación
loss, accuracy, recall, precision, f1 = model.evaluate(validation_generator)
print(f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1-Score: {f1}")

# Guardar el modelo entrenado
model.save(model_path)
print("Modelo almacenado en:", model_path)

Found 2527 images belonging to 23 classes.
Found 803 images belonging to 23 classes.
Cargando modelo existente...
Epoch 1/2
79/79 [==============================] - ETA: 0s - loss: 1.4326 - accuracy: 0.5540 - recall: 0.3455 
Epoch 1: val_accuracy improved from -inf to 0.66625, saving model to /content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/Modelo/food_classification_model_5.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


79/79 [==============================] - 2323s 29s/step - loss: 1.4326 - accuracy: 0.5540 - recall: 0.3455 - val_loss: 1.1515 - val_accuracy: 0.6663 - val_recall: 0.4508
Epoch 2/2
79/79 [==============================] - ETA: 0s - loss: 1.4125 - accuracy: 0.5734 - recall: 0.3593 
Epoch 2: val_accuracy did not improve from 0.66625
26/26 [==============================] - 533s 20s/step - loss: 1.1515 - accuracy: 0.6663 - recall: 0.4508


ValueError: not enough values to unpack (expected 5, got 3)

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from google.colab import drive

# Montar Google Drive
#drive.mount('/content/drive')

# Ruta al modelo entrenado y a la carpeta de predicciones
model_path = '/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/Modelo/food_classification_model_5.h5'
prediction_folder ='/content/drive/MyDrive/Vision Computacional/TRABAJO FINAL/prediccion'

# Verifique si el prediction_folder existe
if not os.path.exists(prediction_folder):
    os.makedirs(prediction_folder)

# Cargar el modelo entrenado
model = load_model(model_path)

# Diccionario de clases ajustado según tus clases
class_indices = {
    'Aguadito de pollo': 0, 'Aji de pollo y arroz': 1, 'Anticucho': 2, 'Arroz chaufa de pollo': 3,
    'Arroz con leche': 4, 'Arroz con pollo': 5, 'Causa rellena': 6, 'Cebiche de pescado': 7,
    'Chanfainita': 8, 'Choclo con queso': 9, 'Lomo saltado con arroz': 10, 'Mazamorra morada': 11,
    'Pan con chicharron': 12, 'Pan con pollo': 13, 'Papa a la huancaína': 14, 'Pollo a la brasa con papas fritas': 15,
    'Pollo broaster con papas fritas': 16, 'Sopa seca con carapulcra': 17, 'Tacacho con cecina': 18,
    'Tallarines rojos con pollo': 19, 'Tallarines saltados con pollo': 20, 'Tallarines verdes con bistec': 21,
    'Tamal de pollo': 22
}
class_labels = {v: k for k, v in class_indices.items()}  # Invertir el diccionario

# Función para preprocesar la imagen
def preprocess_image(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalizar la imagen
    return image

# Leer y predecir cada imagen en la carpeta de predicciones
for file_name in os.listdir(prediction_folder):
    image_path = os.path.join(prediction_folder, file_name)
    image = preprocess_image(image_path)

    # Realizar la predicción
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    confidence = np.max(prediction) * 100  # Calcular el porcentaje de confianza

    # Mostrar la imagen con la etiqueta predicha y el porcentaje de confianza
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(f"Predicción: {predicted_label} - Confianza: {confidence:.2f}%")
    plt.axis('off')
    plt.show()

Output hidden; open in https://colab.research.google.com to view.